In [1]:
# conda install -c conda-forge openblas

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks

#All these packages need to be installed from pip
import requests #for http requests
import pandas as pd #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import numpy as np #For divergences/distances
import scipy #For divergences/distances
import seaborn as sns #makes our plots look nicer
import sklearn.manifold #For a manifold plot
import json #For API responses
import urllib.parse #For joining urls

# comp-linguistics
import spacy

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile

from transformers import pipeline

In [10]:
# nlp = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


21-03-01.txt           corona-21-03.tar       corona.code-workspace
21-05-01.txt           corona-21-05/          corona_testing.ipynb
Corona_to_csv.ipynb    corona-21-05.csv       smaller_data.csv
Corona_to_csv.py       corona-21-05.tar       text-21-05/
all_corona_data.csv    corona-21-08.tar       wlp-21-05/
corona-21-03/          corona-21-09.tar


In [5]:
df = pd.read_csv('all_corona_data.csv')
# df = pd.read_csv('/Users/gabrielnicholson/Desktop/corona/corona-21-05.csv')
# df = pd.read_csv('smaller_data.csv')

In [4]:
df.head()

NameError: name 'df' is not defined

In [32]:
nlp = spacy.load("en_core_web_sm")

In [24]:
df.dropna(axis=0, inplace=True)

In [25]:
df['body_split'] = df['body'].apply(str.split)

In [28]:
def clean_raw_text(text):
    clean_texts = text.replace(" \'m", "'m").replace(" \'ll", "'ll").replace(" \'re", "'re").replace(" \'s", "'s").replace(" \'re", "'re").replace(" n\'t", "n't").replace(" \'ve", "'ve").replace(" /'d", "'d").replace("<h>", "").replace(' Gov ', '')
    return clean_texts

In [29]:
df['body'] = df['body'].apply(lambda x : clean_raw_text(x))

In [30]:
def word_tokenize(word_list):
    tokenized = []
    # pass word list through language model.
    doc = nlp(word_list)
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            tokenized.append(token.text)
    return tokenized

In [35]:
nlp = pipeline("sentiment-analysis", )
result = nlp("We note with concern the proposal by some members of the National Assembly to remove five magistrates of El Salvador's Western Hemisphere Affairs")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
result = nlp("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


label: POSITIVE, with score: 0.8421
label: POSITIVE, with score: 0.9999


In [36]:
def remove_empty(row):
    sentences = []
    for sentence in row:
        if len(sentences) > 500:
            return sentences
        if len(sentence) > 2:
            sentences.append(sentence)
    return sentences

In [12]:
df['sentences'] = df['body'].apply(lambda x: x.split('.'))
df['sentences'] = df['sentences'].apply(remove_empty)

In [39]:
df['sentiment_score'] = df['sentences'].apply(lambda x: nlp(x[:1]))

In [40]:
df.sentiment_score

0       [{'label': 'POSITIVE', 'score': 0.995067834854...
1       [{'label': 'NEGATIVE', 'score': 0.958315372467...
2       [{'label': 'NEGATIVE', 'score': 0.998769581317...
3       [{'label': 'POSITIVE', 'score': 0.624932587146...
4       [{'label': 'POSITIVE', 'score': 0.990642249584...
                              ...                        
1225    [{'label': 'NEGATIVE', 'score': 0.998137593269...
1226    [{'label': 'NEGATIVE', 'score': 0.550629079341...
1227    [{'label': 'POSITIVE', 'score': 0.962613761425...
1228    [{'label': 'NEGATIVE', 'score': 0.999480426311...
1229    [{'label': 'NEGATIVE', 'score': 0.949446082115...
Name: sentiment_score, Length: 1230, dtype: object

In [21]:
df['snippet'][:100].apply(nlp)

0     [{'label': 'NEGATIVE', 'score': 0.873109281063...
1     [{'label': 'NEGATIVE', 'score': 0.971332013607...
2     [{'label': 'NEGATIVE', 'score': 0.966135561466...
3     [{'label': 'POSITIVE', 'score': 0.956532120704...
4     [{'label': 'POSITIVE', 'score': 0.997475683689...
                            ...                        
95    [{'label': 'POSITIVE', 'score': 0.997050404548...
96    [{'label': 'POSITIVE', 'score': 0.989696264266...
97    [{'label': 'NEGATIVE', 'score': 0.960835397243...
98    [{'label': 'NEGATIVE', 'score': 0.953945457935...
99    [{'label': 'NEGATIVE', 'score': 0.999012708663...
Name: snippet, Length: 100, dtype: object

In [47]:
def calc_sentiment(row):
    neg = 0
    pos = 0
    for sentiment in row:
        if sentiment['score'] > 0.80:
            if sentiment['label'] == 'NEGATIVE':
                neg += 1
            else: 
                pos += 1 
        
    return neg/(neg+pos + 1)

In [48]:
df['neg_score'] = df.sentiment_score.apply(calc_sentiment)

In [49]:
df.neg_score

0       0.0
1       0.5
2       0.5
3       0.0
4       0.0
       ... 
1225    0.5
1226    0.0
1227    0.0
1228    0.5
1229    0.5
Name: neg_score, Length: 1230, dtype: float64

In [ ]:
# Creating full sentences and truncating to first 9 sentences in each article.

def remove_empty_sentences(row):
    sentences = []
    for sentence in row:
        if (len(sentence)) < 500 and len(sentence) > 8:
            sentences.append(sentence)
    return sentences

df['sentences'] = df['body'].apply(lambda x: x.split('.'))
df['sentences'] = df['sentences'].apply(lambda x: remove_empty_sentences(x))
df['trunc_sentence'] = df['sentences'].apply(lambda x: x[1:9])

country_df = df[df['country'].apply(lambda x: x in ['US', 'IN', 'CA'])]

from transformers import pipeline
nlp = pipeline("sentiment-analysis")

df['sentiment_score'] = df['trunc_sentence'].apply(lambda x: nlp(x)